In [7]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import statsmodels.api as sm

df = pd.read_excel('raw/MRI_MASTER aktualisiert 020323_de.xlsx')
df = df.drop(columns='CerebralWhiteMatter') # exclude white matter

features = df. columns[8:]

res = pd.DataFrame(index=features)

for feature in features:
    a = df.loc[df.group == 'GAD', feature]
    b = df.loc[df.group == 'HC', feature]
    
    p = stats.ttest_ind(a, b).pvalue
    
    cohens_d = (np.mean(a) - np.mean(b)) / (np.sqrt((np.std(a) ** 2 + np.std(b) ** 2) / 2))
    
    res.loc[res.index==feature, 'mean_hc'] = np.mean(b)
    res.loc[res.index==feature, 'sd_hc'] = np.std(b)
    res.loc[res.index==feature, 'mean_gad'] = np.mean(a)
    res.loc[res.index==feature, 'sd_gad'] = np.std(a)
    res.loc[res.index==feature, 'p_gad_hc'] = p
    res.loc[res.index==feature, 'cohensd_gad_hc'] = cohens_d
    
    for subgroup in df.subgroup.unique():
        subset = df.loc[df.subgroup==subgroup, :]
        
        a = subset.loc[subset.group == 'GAD', feature]
        b = subset.loc[subset.group == 'HC', feature]
        
        p = stats.ttest_ind(a, b).pvalue
        
        cohens_d = (np.mean(a) - np.mean(b)) / (np.sqrt((np.std(a) ** 2 + np.std(b) ** 2) / 2))
        
        res.loc[res.index==feature, f'{subgroup}_mean_hc'] = np.mean(b)
        res.loc[res.index==feature, f'{subgroup}_sd_hc'] = np.std(b)
        res.loc[res.index==feature, f'{subgroup}_mean_gad'] = np.mean(a)
        res.loc[res.index==feature, f'{subgroup}_sd_gad'] = np.std(a)
        res.loc[res.index==feature, f'{subgroup}_p_gad_hc'] = p
        res.loc[res.index==feature, f'{subgroup}_p_gad_hc_sig'] = 1 if p < 0.05 else 0
        res.loc[res.index==feature, f'{subgroup}_cohensd_gad_hc'] = cohens_d

# adjust p vals
p_adj = sm.stats.multipletests(res.p_gad_hc, method='bonferroni')[1]
res.insert(5, 'p_gad_hc_bonferroni', p_adj)

for subgroup in df.subgroup.unique():
    pvals = res[f'{subgroup}_p_gad_hc']
    pvalsadj = sm.stats.multipletests(pvals, method='bonferroni')[1]
    res[f'{subgroup}_p_gad_hc_adj'] = pvalsadj

res = res.sort_values(by='cohensd_gad_hc')

res.to_excel('out/pro_gadmri_results.xlsx')